# Dependências

In [17]:
!pip install boruta

  Using cached Boruta-0.3-py3-none-any.whl (56 kB)


In [18]:
#Importando Bibliotecas
#importantndo bibliotecas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cloudpickle
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

In [2]:
#Constantes.
DATA_RAW_PATH = os.path.join('..','data','raw')
DATA_INTER_PATH = os.path.join('..','data','inter') 
DATA_INTER_TRAIN_NAME = 'train_inter.csv'
IMG_PATH = os.path.abspath(os.path.join('..','img'))
META_DATA_PATH = os.path.join(DATA_RAW_PATH,'dicionario_tipos.pkl')

In [3]:
# Dados
with open(META_DATA_PATH,'rb') as f:
    dicionario_tipo = cloudpickle.load(f)
df_train = pd.read_csv(os.path.join(DATA_INTER_PATH, DATA_INTER_TRAIN_NAME), dtype=dicionario_tipo)

In [4]:
df_train.head()

,Department,Role,Will_Relocate,Trending Perf,Talent_Level,Percent_Remote,EMP_Sat_OnPrem_1,EMP_Sat_OnPrem_2,EMP_Sat_OnPrem_3,EMP_Sat_OnPrem_5,EMP_Sat_Remote_1,EMP_Sat_Remote_2,EMP_Sat_Remote_3,EMP_Sat_Remote_4,EMP_Sat_Remote_5,EMP_Engagement_1,EMP_Engagement_2,EMP_Engagement_3,EMP_Engagement_4,EMP_Engagement_5,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left_Company,promotion_last_5years,sales,salary,LinkedIn_Hits,Emp_Work_Status2,Emp_Work_Status_3,Emp_Work_Status_4,Emp_Work_Status_5,Emp_Identity,Emp_Role,Emp_Position,Emp_Title,Emp_Competitive_1,Emp_Competitive_2,Emp_Competitive_3,Emp_Competitive_4,Emp_Competitive_5,Emp_Collaborative_1,Emp_Collaborative_2,Emp_Collaborative_3,Emp_Collaborative_4,Emp_Collaborative_5,Sensor_StepCount,Sensor_Heartbeat(Average/Min),Sensor_Proximity(1-highest/10-lowest)
0,Finance,Manager,1,3,6,0.8,3.0,3.0,4.0,4.0,3,5,4,4,5,5,5,4,4,3,0.90,3,138,3,0,0,0,marketing,medium,10,4,3,4,1,2,4,2,2,4,1,4,3,5,4,2,1,3,4,4327,76,8
1,Sales,Level 2-4,0,3,6,0.8,3.0,3.0,4.0,4.0,3,5,4,4,5,5,5,4,4,3,0.95,3,251,3,0,0,0,support,low,1,5,1,1,4,4,2,3,2,2,5,2,5,3,5,2,2,3,4,5073,78,4
2,Human Resources,Level 1,0,3,6,0.8,3.0,3.0,4.0,4.0,3,5,4,4,5,5,5,4,4,3,0.82,4,134,6,0,0,0,technical,medium,7,5,2,5,5,4,4,4,3,4,1,3,5,4,5,4,4,4,1,4881,70,6
3,Warehouse,Director,0,3,6,0.4,3.0,3.0,4.0,5.0,2,3,2,3,2,5,5,4,4,3,0.46,2,186,2,0,0,0,product_mng,low,45,4,2,1,4,2,1,1,1,5,4,5,1,5,5,1,1,1,3,3234,86,8
4,Sales,Level 2-4,0,3,6,0.4,3.0,3.0,4.0,5.0,2,3,2,3,2,5,5,4,4,3,0.47,5,185,4,0,0,0,product_mng,medium,32,4,3,1,2,1,1,2,1,2,3,2,1,1,2,1,5,1,3,3413,88,7


# Tópico de Seleção de Feature.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 51 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Department                             11999 non-null  object 
 1   Role                                   11999 non-null  object 
 2   Will_Relocate                          11999 non-null  int64  
 3   Trending Perf                          11999 non-null  int64  
 4   Talent_Level                           11999 non-null  int64  
 5   Percent_Remote                         11999 non-null  float64
 6   EMP_Sat_OnPrem_1                       11999 non-null  float64
 7   EMP_Sat_OnPrem_2                       11999 non-null  float64
 8   EMP_Sat_OnPrem_3                       11999 non-null  float64
 9   EMP_Sat_OnPrem_5                       11999 non-null  float64
 10  EMP_Sat_Remote_1                       11999 non-null  int64  
 11  EM

NameError: name 'X' is not defined